In [1]:
import itertools as it

def genBooleanPermutations(n:int) -> list[tuple[bool]]:
    """generates all combinations (size 2^n) of trues and falses with n booleans"""
    return list(it.product([True,False], repeat=n))

for n in range(3):
    print(genBooleanPermutations(n))

[()]
[(True,), (False,)]
[(True, True), (True, False), (False, True), (False, False)]


In [17]:
import numpy

class Factor:
    """ A Helper object to execute the `variable elminiation algorithm`"""
    # a factor consists of 2 attributes:
    #    table: a multi dimentsional numpy array where each entrie is a float
    #    variables: an array of the variables associated with the factor
    # EX. consider the factor f(A, B, C), which represents P(A=a|B=b and C=c)
    
    def __init__(self, variables: list, values_dict:dict[tuple, float]):
        # for example, say we want factor f to represent P(A=a | B=b and C=c):
        #   the the input must be:
        #      variables = [A, B, C],
        #      value_dict = {(True, True, True):    P(A=t | B=t and C=t),
        #                    (True, True, False):   P(A=t | B=t and C=f),
        #                    (True, False, True):   P(A=t | B=f and C=t),
        #                    (True, False, False):  P(A=t | B=f and C=f):,
        #                    (False, True, True):   P(A=f | B=t and C=t),
        #                    (False, True, False):  P(A=f | B=t and C=f),
        #                    (False, False, True):  P(A=f | B=f and C=t),
        #                    (False, False, False): P(A=f | B=f and C=f)}
        
        self.variables = variables
        arr = numpy.ndarray(shape=((2,) * len(variables)))
        perms = genBooleanPermutations(len(variables))

        for perm in perms:
            n = len(perm)
            cords = ()
            for i in range(n):
                if(perm[i] == True): 
                    cords += (1,)
                else:
                    cords += (0,)
            arr[cords] = values_dict[perm]
        self.table = arr
    
    def __str__(self):
        return f"\nvariables: {self.variables}\ntable:\n{self.table}"

    def __repr__(self):
        return str(self)

    def get_index(self, value):
        val = 0
        if (value == True) :
            val = 1
        return val
    
    def get_axis(self, variable):
        axis = 0
        n = len(self.variables)
        i = 0
        while(i < n):
            if(self.variables[i] == variable):
                axis = i
                break
            i += 1
        return axis

In [19]:
## Bayesnet Representation
v = ["A", "B", "C"]
vd = {
      (True, True, True): 0.03,
      (True, True, False): 0.07,
      (True, False, True): 0.54,
      (True, False, False): 0.36,
      (False, True, True): 0.06,
      (False, True, False): 0.14,
      (False, False, True): 0.48,
      (False, False, False): 0.32,
}

f_1 = Factor(v, vd)
print(f_1)


variables: ['A', 'B', 'C']
table:
[[[0.32 0.48]
  [0.14 0.06]]

 [[0.36 0.54]
  [0.07 0.03]]]


In [9]:
import copy

def restrict(factor:Factor, variable:str, value:bool):
    """ produce a new factor where one of the variables has been resticted to either true or false."""
    f = copy.deepcopy(factor)
    index = f.get_index(value)
    axis = f.get_axis(variable)
    f.variables.remove(variable)
    f.table = f.table.take(index, axis = axis)
    return f


In [20]:
f_2 = restrict(f_1, "B", False)
print(f_2)

f_2 = restrict(f_2, "C", False)
print(f_2)

f_2 = restrict(f_2, "A", True)
print(f_2)


variables: ['A', 'C']
table:
[[0.32 0.48]
 [0.36 0.54]]

variables: ['A']
table:
[0.32 0.36]

variables: []
table:
0.36


In [23]:
def multiply(factor1:Factor, factor2:Factor)-> Factor:
    f_1 = copy.deepcopy(factor1)
    f_2 = copy.deepcopy(factor2)

    new_vars = []
    new_vars.extend(f_1.variables)
    new_vars.extend(f_2.variables)
    new_vars = list(set(new_vars))
    
    n = len(new_vars) 
    
    where = []
    for var in new_vars:
        if (var in f_1.variables and var in f_2.variables):
            where.append("Common") 
        elif (var in f_1.variables):
            where.append("f_1")
        elif (var in f_2.variables):
            where.append("f_2")
        else :
            print("Cant do this!!!!")
    
    # now we know where to look
    # but what happens when we want to solve for f.table[True, True, True]?
    #  il.e. how will we know which cel of f_1 and f_2 to multiply?
    #  in our example case, new_vars = ['B', 'C', 'A'].  so then,
    #  f.table['B'=True, 'C'=True, 'A'=true] = 
    #      f_1.table.restrict(A = "True").restrict(B = "True") *
    #      f_2.table.restrict(B = "True").resticrt(C = "True")
    
    new_val_dict = {}
    perms = genBooleanPermutations(n)
    for perm in perms:
        # the goal of this loop is to generate the dictonary where
        #   the key is all the permutations of trues and falses of length
        #   len(new_var), and the value is the number specfied by the factor 
        #   multiply rule.  

        f_3 = copy.deepcopy(factor1)
        f_4 = copy.deepcopy(factor2)
        
        for i in range(n):
            info = where[i]
            if(info == "Common" or info == "f_1"):
                f_3 = restrict(f_3, new_vars[i], perm[i])
            if(info == "Common" or info == "f_2"):
                f_4 = restrict(f_4, new_vars[i], perm[i])
        prob = f_3.table * f_4.table
        #new_val_dict[perm] = round(prob, 2)
        new_val_dict[perm] = prob
    f = Factor(new_vars, new_val_dict)
    return f

In [24]:
f1 = Factor(
    ["A", "B"],
    {
      (True, True): 0.1,
      (True, False): 0.9,
      (False, True): 0.2,
      (False, False): 0.8,
     }
)

f2 = Factor(
    ["B", "C"],
    {
      (True, True): 0.3,
      (True, False): 0.7,
      (False, True): 0.6,
      (False, False): 0.4,
     }
)

f = multiply(f1, f2)
print(f)


variables: ['B', 'A', 'C']
table:
[[[0.32 0.48]
  [0.36 0.54]]

 [[0.14 0.06]
  [0.07 0.03]]]


In [26]:
def sumout(factor:Factor, variable:str) -> Factor:
    f = copy.deepcopy(factor)
    f.table = numpy.sum(f.table, axis = f.get_axis(variable))
    f.variables.remove(variable)
    return f

f = sumout(f_1, "B")
print(f)


variables: ['A', 'C']
table:
[[0.46 0.54]
 [0.43 0.57]]


In [28]:
def normalize(factor):
    f = copy.deepcopy(factor)
    Z = numpy.sum(factor.table)
    for x in numpy.nditer(f.table, op_flags=['readwrite']):
        x[...] = x / Z
    return f

f_1 = Factor(v, vd)
f = normalize(f_1)
print(f)


variables: ['A', 'B', 'C']
table:
[[[0.16  0.24 ]
  [0.07  0.03 ]]

 [[0.18  0.27 ]
  [0.035 0.015]]]


In [29]:
# an evidence list is a list of variables that are known to be either true or false.
#   evidenceList[i] = ["var_name": Bool] where Bool = {True or False}
def inference(factorList, queryVariables, orderedListOfHiddenVariables, evidenceList):
    # i)
    # Restrict the factors in factorList according to the evidence in evidenceList.
   
    for i in range(len(factorList)):
        for evidence in evidenceList:
            if(evidence[0] in factorList[i].variables):
                factorList[i] = copy.deepcopy(restrict(factorList[i], evidence[0], evidence[1]))

        
    # ii)
    # Based on the order of the hidden variables in orderedListOfHiddenVariables
    #   sum out each hidden variable from the product of the factors in factorList
    
    for hv in orderedListOfHiddenVariables:
        # use this array to keep track of which factors we are multiplying
        new_factor_list = []
        mult = []
        for i in range(len(factorList)):
            if(hv in factorList[i].variables):
                # if the factor contains hv, prep for multiplication
                mult.append(factorList[i])
            else:
                new_factor_list.append(factorList[i])
        if(mult != []):     
            n_mult = len(mult)
            f_prod = copy.deepcopy(mult[0])
            for i in range(1, n_mult):
                f_prod = multiply(f_prod, mult[i])
            f_new = sumout(f_prod, hv)
            new_factor_list.append(f_new)
            factorList = new_factor_list
    
    
    # iii)
    # if the result is a product of multiple factors, 
    #  then multiply them to produce a single factor
    n = len(factorList)
    if(n <= 0):
        print("something wrong")
    result =  factorList[0]
    if(n > 1):
        for i in range(1, n):
            result = multiply(result, factorList[i])
            
    # iv)
    # normalize
    
    result = normalize(result)

    return result

In [30]:
f1 = Factor(
    ["E"],
    {
      (True,): 0.0003,
      (False,): 0.9997 
     }
)

f2 = Factor(
    ["B"],
    {
      (True,): 0.0001,
      (False,): 0.9999 
     }
)

f3 = Factor(
    ["R", "E"],
    {
      (True, True): 0.9,
      (True, False): 0.0002,
      (False, True): 0.1,
      (False, False): 0.9998,
     }
)

f4 = Factor(
    ["A", "B", "E"],
    {
      (True, True, True): 0.96,
      (True, True, False): 0.95,
      (True, False, True): 0.2,
      (True, False, False): 0.01,
      (False, True, True): 0.04,
      (False, True, False): 0.05,
      (False, False, True): 0.8,
      (False, False, False): 0.99,
     }
)

f5 = Factor(
    ["W", "A"],
    {
      (True, True): 0.8,
      (True, False): 0.4,
      (False, True): 0.2,
      (False, False): 0.6,
     }
)

f6 = Factor(
    ["G", "A"],
    {
      (True, True): 0.4,
      (True, False): 0.04,
      (False, True): 0.6,
      (False, False): 0.96,
     }
)

fl = [f1, f2, f3, f4, f5, f6]
el = [["W", True], ["G", True]]
olohv = ["R", "E", "A"]
qv = ["B"]

f = inference(fl, qv, olohv, el)

print(f)


variables: ['B']
table:
[0.998403 0.001597]
